# Resnet walkthrough

Full code can be found [here, Resnet on MNIST](https://github.com/rasbt/deeplearning-models/blob/master/pytorch_ipynb/cnn/cnn-resnet50-mnist.ipynb). I will extract and paste relevant snippets in this nb to illustrate some points regarding batch normalizaition. 

In [ ]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        # motif: conv -> bn -> activation 
        
        out = self.conv1(x)
        out = self.bn1(out) # includes bias 
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

The forward pass structure that is followed in defining the 'identity block' (bbuilding block) in the resnet: 
1. Conv2D layer
2. Batch normalization 
3. Activation 
4. Loop on 1-2-3 as long as blocks exist!

So: It is common to have BN after every linear or conv layer followed by activation. 

- We can also see the `'he_normal'` initialization, which is pytorch argument for __Kaiming normal__ init. 

- ```self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)``` $\rightarrow$ bias  = False, since internally `bnmeani` is being included while normalizing, rendering an explicit bias declaration redundant. 